In [19]:
from pyspark.sql import SparkSession, dataframe
from pyspark.sql.functions import when, col, sum, count, isnan, round
from pyspark.sql.functions import regexp_replace, concat_ws, sha2, rtrim, substring
from pyspark.sql.functions import unix_timestamp, from_unixtime, to_date
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType
from pyspark.sql import HiveContext

import os
import re

from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import when

spark = SparkSession.builder.master("local[*]")\
    .enableHiveSupport()\
    .getOrCreate()

In [20]:
def salvar_df(df, file):
    output = "/input/projeto_hive/gold/" + file
    erase = "hdfs dfs -rm " + output + "/*"
    rename = "hdfs dfs -get /datalake/gold/"+file+"/part-* /input/projeto_hive/gold/"+file+".csv"
    print(rename)
    
    
    df.coalesce(1).write\
        .format("csv")\
        .option("header", True)\
        .option("delimiter", ";")\
        .mode("overwrite")\
        .save("/datalake/gold/"+file+"/")

    os.system(erase)
    os.system(rename)

In [21]:
df_clientes = spark.sql("select * from desafio_curso.clientes")
df_divisao = spark.sql("select * from desafio_curso.divisao")
df_endereco = spark.sql("select * from desafio_curso.endereco")
df_regiao = spark.sql("select * from desafio_curso.regiao")
df_vendas = spark.sql("select * from desafio_curso.vendas")

In [25]:
df_stage_clientes_localidade = df_clientes.join(df_endereco, "address_number", "inner")\
                                .join(df_divisao, "division", "inner")\
                                .join(df_regiao, "region_code", "inner")\
                                .join(df_vendas, "customer_key", "inner")

In [26]:
df_stage_clientes_localidade.show()

+------------+-----------+--------+--------------+--------------------+-------------+-----------+-------------+----------------+------------+------------------+-----------+--------+-------+-------------------+--------------------+--------------------+--------------------+-----+--------+-------------+-----------+--------------------+----------+---------------+------------+--------------+----------+-----------+--------------------+-----------+----------+------------+----------------------+------------+--------------------------------+-----------------+-------------------+-----------+--------------+---------+---+
|customer_key|region_code|division|address_number|business_family_name|business_unit|   customer|customer_type|line_of_business|       phone|regional_sales_mgr|search_type|    city|country| customer_address_1|  customer_address_2|  customer_address_3|  customer_address_4|state|zip_code|division_name|region_name|actual_delivery_date|  date_key|discount_amount|invoice_date|invoice_

In [24]:
df_vendas.show()

+--------------------+------------+----------+---------------+------------+--------------+----------+-----------+--------------------+-----------+----------+------------+----------------------+------------+--------------------------------+-----------------+-------------------+-----------+--------------+---------+---+
|actual_delivery_date|customer_key|  date_key|discount_amount|invoice_date|invoice_number|item_class|item_number|                item|line_number|list_price|order_number|promised_delivery_date|sales_amount|sales_amount_based_on_list_price|sales_cost_amount|sales_margin_amount|sales_price|sales_quantity|sales_rep|u_m|
+--------------------+------------+----------+---------------+------------+--------------+----------+-----------+--------------------+-----------+----------+------------+----------------------+------------+--------------------------------+-----------------+-------------------+-----------+--------------+---------+---+
|Actual Delivery Date| CustomerKey|   DateK